In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [2]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [3]:
main_df = pd.read_csv('spacex_launch_geo.csv')
main_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
spacex_df = main_df[['Launch Site','Lat','Long','class']]
launch_sites_df = main_df[['Launch Site','Lat','Long']].groupby('Launch Site').first().reset_index()
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
marker_color_list = []

for outcome in spacex_df['class']:
    if outcome==1:
        marker_color_list.append('green')
    else:
        marker_color_list.append('red')

spacex_df['marker_color'] = marker_color_list
spacex_df.head(20)

C:\Users\-\AppData\Local\Temp\ipykernel_23948\2188638085.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['marker_color'] = marker_color_list


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


In [6]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(nasa_coordinate,
                           icon=DivIcon(icon_size=(20,20),
                           icon_anchor=(0,0),
                           html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % 'NASA JSC'))
site_map.add_child(circle)
site_map.add_child(marker)


for i in range(0, len(launch_sites_df['Launch Site'])):
    
    loc = [launch_sites_df.iloc[i, 1], launch_sites_df.iloc[i, 2]]

    circle = folium.Circle(loc, radius=500, color='#d35400', fill=True).add_child(folium.Popup(launch_sites_df.iloc[i, 0]))
    marker = folium.map.Marker(loc,
                               icon=DivIcon(icon_size=(20,20),
                               icon_anchor=(0,0),
                               html='<div style=\"font-size: 12; color:#d35400;\"><b>%s</b></div>' % launch_sites_df.iloc[i, 0]))
    
    site_map.add_child(circle)
    site_map.add_child(marker)

    marker_cluster = MarkerCluster()
    site_map.add_child(marker_cluster)
    
    site_df = spacex_df[spacex_df['Launch Site']==launch_sites_df.iloc[i, 0]].reset_index(drop=True)

    for j in range(0, len(site_df)):
        
        marker = folium.map.Marker(loc,
                                   icon=folium.Icon(color=site_df.iloc[j, 4]))
        marker_cluster.add_child(marker)

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,)
site_map.add_child(mouse_position)

marker = folium.map.Marker([28.56367, -80.568],icon=folium.Icon(color='blue'))
site_map.add_child(marker)

distance = round(calculate_distance(28.563197,-80.576820, 28.56367,-80.568),2)

distance_marker = folium.Marker([28.56367, -80.568],
                                icon=DivIcon(icon_size=(20,20),
                                icon_anchor=(0,0),
                                html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' %"{:10.2f} KM".format(distance)))
site_map.add_child(distance_marker)

lines=folium.PolyLine(locations=[[28.563197,-80.576820],[28.56367,-80.568]], weight=1)
site_map.add_child(lines)

site_map